# Generate experimental features -  Cell 1 - 010321

In [ ]:
import json
import matplotlib.pyplot as plt

import bluepyopt as bpopt
import bluepyopt.ephys as ephys

import model
import evaluator
import time
import neuron
import plotting
import MEAutility as mu
from pprint import pprint
import numpy as np

import sys
import shutil

from pathlib import Path
import os

%matplotlib notebook

## 1) Generate features and protocols with BluePyEfe

In [ ]:
sys.path.append('../efeatures_extraction')

In [ ]:
data_folder = Path("../data/experimental/cell1_210301/")

In [ ]:
extra_folder = data_folder / "extracellular"

In [ ]:
with (extra_folder / "probe.json").open() as f:
    probe_info = json.load(f)
    probe_info["pitch"] = 17.5
    probe = mu.return_mea(info=probe_info)

In [ ]:
eap = np.load(extra_folder / "template.npy")
locations = np.load(extra_folder / "locations.npy")
fs = 20000

In [ ]:
mu.plot_mea_recording(eap, probe)

In [ ]:
efeatures_output_directory = Path(f"../data/experimental/cell1_210301/efeatures")
ephys_dir = Path("../data/experimental/cell1_210301/patch_data/")

In [ ]:
from bluepyefe.extract import read_recordings, extract_efeatures_at_targets, compute_rheobase,\
    mean_efeatures, create_feature_protocol_files
from bluepyefe.plotting import plot_all_recordings_efeatures

from extraction_tools import build_wcp_metadata, wcp_reader, get_targets, ecodes_wcp_timings

In [ ]:
# select files for different repetitions

In [ ]:
runs = [2, 3, 4]  # run1 --> different rheobase

ecode_to_index = {
    "IDthres": 0,
    "firepattern": 1,
    "IV": 2,
    "IDrest": 3,
    "APWaveform": 4,
    "HyperDepol": 5,
    "sAHP": 6,
    "PosCheops": 7
}

files_list = []

for run in runs:
    rep_dict = {}
    for ecode in ecode_to_index:
        rep_dict[ecode] = Path(ephys_dir) / f"cell1_run{run}.{ecode_to_index[ecode]}.wcp"
    files_list.append(rep_dict)

In [ ]:

# define timings for this experiment
ecodes_cell1_timings = {
    "IDthres": {
        'ton': 200,
        'toff': 470
    },
    "firepattern": {
        'ton': 500,
        'toff': 4100
    },
    "IV": {
        'ton': 250,
        'toff': 3250
    },
    "IDrest": {
        'ton': 200,
        'toff': 1550
    },
    "APWaveform": {
        'ton': 150,
        'toff': 200
    },
    "HyperDepol": {
        'ton': 200,
        'toff': 920,
        'tmid': 650
    },
    "sAHP": {
        'ton': 200,
        'toff': 1125,
        'tmid': 450,
        'tmid2': 675
    },
    "PosCheops": {
        'ton': 1000,
        't1': 9000,
        't2': 10500,
        't3': 14500,
        't4': 16000,
        'toff': 18660
    }
}

In [ ]:
files_list

In [ ]:
files_metadata = build_wcp_metadata(cell_id="cell1_010321", files_list=files_list, 
                                    ecode_timings=ecodes_cell1_timings, 
                                    repetition_as_different_cells=False)
pprint(files_metadata["cell1_010321"])

In [ ]:
cells = read_recordings(
    files_metadata=files_metadata,
    recording_reader=wcp_reader
)

In [ ]:
# define target features for different protocols
targets = get_targets(ecodes_cell1_timings)

In [ ]:
targets["firepattern"]["tolerances"] = [20, 40]
targets["IDrest"]["tolerances"] = [20]
targets["PosCheops"]["tolerances"] = [50]
targets["HyperDepol"]["tolerances"] = [40]
targets["APWaveform"]["tolerances"] = [40]
targets["sAHP"]["tolerances"] = [40]

In [ ]:
pprint(targets)

In [ ]:
t_start = time.time()
extract_efeatures_at_targets(
    cells, 
    targets,
)
t_stop = time.time()
print(f"Elapsed time {t_stop - t_start}")

In [ ]:
compute_rheobase(
    cells, 
    protocols_rheobase=['IDthres']
)

In [ ]:
print(f"Cell rheobase: {cells[0].rheobase}")

In [ ]:
for cell in cells:
    for recording in cell.recordings:
        if recording.protocol_name == "sAHP":
            print(recording.protocol_name, recording.amp_rel, recording.amp2_rel)
        else:
            print(recording.protocol_name, recording.amp_rel)

In [ ]:
targets

In [ ]:
protocols = mean_efeatures(cells, targets, use_global_rheobase=True)

In [ ]:
len(protocols)

In [ ]:
efeatures, protocol_definitions, current = create_feature_protocol_files(
    cells,
    protocols,
    output_directory=efeatures_output_directory,
    threshold_nvalue_save=1,
    write_files=False,
)

In [ ]:
# make df
protocol_array = []
feat_names = []
feat_values = []

for k, v in efeatures.items():
    for feat in v["soma"]:
        protocol_array.append(k) 
        feat_names.append(feat["feature"])
        feat_values.append(feat["val"][0])        

In [ ]:
import pandas as pd
import seaborn as sns

In [ ]:
df = pd.DataFrame({"protocol": protocol_array, "name": feat_names, "value": feat_values})

In [ ]:
f, ax = plt.subplots()
sns.barplot(data=df, x="name", y="value")
ax.set_xticklabels([])
ax.set_xlabel("Intracellular features", fontsize=15)
ax.set_ylabel("Values", fontsize=15)
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)

In [ ]:
efeatures["firepattern_120"]

## 3) Convert to BPO format and append extra features

In [ ]:
from extraction_tools import convert_to_bpo_format, append_extrafeatures_to_json, compute_extra_features

In [ ]:
protocols_of_interest = ["firepattern_200", "IV_-100", "APWaveform_260"]

in_protocol_path = efeatures_output_directory / "protocols.json"
in_efeatures_path = efeatures_output_directory / "features.json"

out_protocol_path = efeatures_output_directory / "protocols_BPO.json"
out_efeatures_path = efeatures_output_directory / "features_BPO.json"

In [ ]:
protocols_dict, efeatures_dict = convert_to_bpo_format(in_protocol_path, in_efeatures_path, 
                                                       out_protocol_path, out_efeatures_path, 
                                                       protocols_of_interest=protocols_of_interest, 
                                                       std_from_mean=0.2)

In [ ]:
pprint(efeatures_dict)

In [ ]:
extra_features = compute_extra_features(eap, fs, upsample=10)

In [ ]:
pprint(extra_features.keys())

In [ ]:
efeatures_dict = append_extrafeatures_to_json(extra_features, protocol_name="firepattern_200",
                                              efeatures_path=out_efeatures_path)

In [ ]:
pprint(efeatures_dict["extra"]["firepattern_200"])

In [ ]:
# plot one extra features
plotting.plot_feature_map(extra_features["pos_peak_diff"], probe, bg=False)

# Threshold EAP, compute extra features and save probe json file

In [ ]:
def plot_feature_w_colorbar(extra_features, feat_name, label, height_ratio=[10, 1]):
    import matplotlib as mpl
    
    fig, axs = plt.subplots(
        nrows=2, ncols=1, 
        gridspec_kw={'height_ratios': height_ratio}
        )

    plotting.plot_feature_map(extra_features_above[feat_name], probe_above, bg=False, ax=axs[0])

    cmap = plt.get_cmap("viridis")
    norm = mpl.colors.Normalize(vmin=np.min(extra_features_above[feat_name]), 
                                vmax=np.max(extra_features_above[feat_name]))

    cb1 = mpl.colorbar.ColorbarBase(axs[1], cmap=cmap,
                                    norm=norm,
                                    orientation='horizontal')
    
    cb1.set_label(label)
        
    fig.suptitle(feat_name, fontsize=15)
    
    return fig

In [ ]:
extra_features_above.keys()

In [ ]:
thresh_uV = 5

In [ ]:
amp_eap = np.ptp(eap, 1)
above_tr = np.where(amp_eap > thresh_uV)
eap_above = eap[above_tr]
extra_features_above = compute_extra_features(eap_above, fs, upsample=10)
probe_info = probe.info
probe_info["pos"] = locations[above_tr]
probe_above = mu.return_mea(info=probe_info)

In [ ]:
for k, v in extra_features_above.items():
    print(k, min(v), max(v))

In [ ]:
plt.figure()
plt.plot(eap_above.T, "k", alpha=0.5)
plt.plot(eap_above[219], "C1", alpha=1)

In [ ]:
f = plot_feature_w_colorbar(extra_features_above, "neg_image", 'Rel. Amplitude')
f = plot_feature_w_colorbar(extra_features_above, "pos_image", 'Rel. Amplitude')
f = plot_feature_w_colorbar(extra_features_above, "neg_peak_diff", 'time (s)')
f = plot_feature_w_colorbar(extra_features_above, "peak_to_valley", 'time (s)')

In [ ]:
fig, axs = plt.subplots(
        nrows=2, ncols=1, 
        gridspec_kw={'height_ratios': [10, 1]}
        )

plotting.plot_feature_map(extra_features_above["neg_image"], probe_above, bg=False, ax=axs[0])

cmap = plt.get_cmap("viridis")
norm = mpl.colors.Normalize(vmin=np.min(extra_features_above["neg_image"]), 
                            vmax=np.max(extra_features_above["neg_image"]))

cb1 = mpl.colorbar.ColorbarBase(axs[1], cmap=cmap,
                                norm=norm,
                                orientation='horizontal')
cb1.set_label('Amplitude (uV)')

In [ ]:
extra_features_above["neg_image"]

In [ ]:
plotting.plot_feature_map(extra_features_above["pos_image"], probe_above, bg=False)

In [ ]:
plotting.plot_feature_map(extra_features_above["neg_image"], probe_above, bg=False)

In [ ]:
plt.figure()
_ = plt.plot(eap[above_tr].T, "k")

In [ ]:
import bluepyopt.ephys as ephys

In [ ]:
morph = ephys.morphologies.NrnFileMorphology("../data/experimental/cell1_210301/morphology_corrected.swc")

In [ ]:
sim = ephys.simulators.NrnSimulator()

In [ ]:
morph.instantiate(sim=sim)

In [ ]:
seclist_names = ['all', 'somatic', 'basal', 'apical', 'axon_initial_segment', 'hillockal', 'myelinated', 'axonal']
secarray_names = ['soma', 'dend', 'apic', 'axon', 'ais', 'hillock', 'myelin' ]

In [ ]:
cell = ephys.models.LFPyCellModel(
        "cell1",
        v_init=-70,
        morph=morph,
        mechs=None,
        params=None,
        seclist_names=seclist_names,
        secarray_names=secarray_names
    )

In [ ]:
import LFPy
import neuron
import neuroplotlib as nplt

In [ ]:
cell_lfpy = LFPy.Cell(morphology="../data/experimental/cell1_210301/morphology_corrected.swc",
                      pt3d=True)

In [ ]:
cell_lfpy.allsecnames

In [ ]:
delete_section_name = "dend_7"
rename_section_name = "ais[0]"

sec_renamed = neuron.h.Section(name=rename_section_name)
for sec in icell.allsec():
    if delete_section_name in sec.name():
        n3d = sec.n3d()
        for i in range(n3d):
            diams.append(sec.diam3d(i))
            pt3d.append([sec.x3d(i), sec.y3d(i), sec.z3d(i)])
            neuron.h.pt3dadd(sec.x3d(i), sec.y3d(i), sec.z3d(i), sec.diam3d(i), sec=sec_renamed)  
        parentseg = sec.parentseg()
        children = sec.children()
        neuron.h.pt3dclear(sec=sec)
        neuron.h.delete_section(sec=sec)
        break

sec_renamed.connect(parentseg.sec, 1.0, 0.0)
for child in children:
    child.connect(sec_renamed, 1.0, 0.0)

cell_lfpy2 = LFPy.Cell(morphology=neuron.h.allsec(), pt3d=True)

In [ ]:
for sec in neuron.h.allsec():
    print(sec.name())

In [ ]:
for sec in cell_lfpy2.allseclist:
    print(sec.name())

In [ ]:
%matplotlib notebook

In [ ]:
nplt.plot_detailed_neuron(cell_lfpy2, color_ais="g", plane="xy")

In [ ]:
cell_lfpy2.allsecnames

In [ ]:
cell_lfpy.get_idx_parent_children('dend_7[0]')

In [ ]:
sec_ais = neuron.h.Section(rename_section_name)

In [ ]:
cell_lfpy.get_idx_parent_children?

In [ ]:
sec_ais.trueparentseg()

In [ ]:
#we erase the 3d info from the section and replace with the new interpolated 3d info
for i in range(len(pt3d)):
    neuron.h.pt3dadd(pt3d[i][0], pt3d[i][1], pt3d[i][2], diams[i], sec=sec_ais)  

In [ ]:
for sec in cell_lfpy.allseclist:
    print(sec.name())